In [33]:
import re
from collections import defaultdict
from operator import itemgetter
from pprint import pp
from queue import PriorityQueue
from sys import maxsize

import numpy as np

second = itemgetter(1)

sample = """Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi"""

def get_array(data):
    width = len(data.splitlines()[0])
    digits = re.findall(r'[SEa-z]', data)
    digits = [ord(c) for c in digits]
    array = np.array(digits, dtype=int)
    array.shape = len(digits) // width, width
    return array


def extract_start_stop(array):
    start = np.where(array==83)
    array[start] = 97
    stop = np.where(array==69)
    array[stop] = 122
    return (start[0][0], start[1][0]), (stop[0][0], stop[1][0])


array = get_array(sample)

start, stop = extract_start_stop(array)
print(start, stop)

(0, 0) (2, 5)


In [90]:

def get_neighbors(array):
    debug_results = {}
    forward = {}
    backward = {}
    for y, row in enumerate(array):
        for x, col in enumerate(row):
            neighbors = []
            if y > 0:
                neighbors.append((y - 1,x))
            if y < array.shape[0] - 1:
                neighbors.append((y + 1,x))
            if x > 0:
                neighbors.append((y,x - 1))
            if x < array.shape[1] - 1:
                neighbors.append((y,x + 1))
            forward[(y,x)] = {n: col - array[n] + 2 for n in neighbors if col >= array[n] - 1}
            backward[(y,x)] = {n: array[n] - col + 2 for n in neighbors if array[n] >= col - 1}
            debug_results[(chr(col),col,y,x)] = {n: (chr(array[n]),col - array[n] + 2) for n in neighbors if col >= array[n]-1}
    return forward, debug_results, backward




array = get_array(sample)
start,stop = extract_start_stop(array)
forward,debug_results,backward = get_neighbors(array)
from pprint import pp
print(f'{start=} {stop=}')
pp(debug_results)

                


start=(0, 0) stop=(2, 5)
{('a', 97, 0, 0): {(1, 0): ('a', 2), (0, 1): ('a', 2)},
 ('a', 97, 0, 1): {(1, 1): ('b', 1), (0, 0): ('a', 2), (0, 2): ('b', 1)},
 ('b', 98, 0, 2): {(1, 2): ('c', 1), (0, 1): ('a', 3)},
 ('q', 113, 0, 3): {(1, 3): ('r', 1), (0, 2): ('b', 17), (0, 4): ('p', 3)},
 ('p', 112, 0, 4): {(0, 3): ('q', 1), (0, 5): ('o', 3)},
 ('o', 111, 0, 5): {(0, 4): ('p', 1), (0, 6): ('n', 3)},
 ('n', 110, 0, 6): {(0, 5): ('o', 1), (0, 7): ('m', 3)},
 ('m', 109, 0, 7): {(1, 7): ('l', 3), (0, 6): ('n', 1)},
 ('a', 97, 1, 0): {(0, 0): ('a', 2), (2, 0): ('a', 2), (1, 1): ('b', 1)},
 ('b', 98, 1, 1): {(0, 1): ('a', 3),
                   (2, 1): ('c', 1),
                   (1, 0): ('a', 3),
                   (1, 2): ('c', 1)},
 ('c', 99, 1, 2): {(0, 2): ('b', 3), (2, 2): ('c', 2), (1, 1): ('b', 3)},
 ('r', 114, 1, 3): {(0, 3): ('q', 3), (2, 3): ('s', 1), (1, 2): ('c', 17)},
 ('y', 121, 1, 4): {(0, 4): ('p', 11),
                    (2, 4): ('z', 1),
                    (1, 3): ('r', 9

In [91]:




class PathFinder:
    def __init__(self, data, debug=False):
        self.debug = debug
        if isinstance(data, str):
            self.array = get_array(data)
        else:
            self.array = data
        self.start, self.stop = extract_start_stop(self.array)
        self.neighbors, _, self.backward = get_neighbors(self.array)

    def get_todo(self):
        while True:
            if not self.todo.empty():
                ondeck = self.todo.get()
                yield second(ondeck)
            else:
                break

    def get_path_to_position(self, next_hop):
        hops = []
        while True:
            neighbors = sorted(self.backward[next_hop], key=lambda n: self.knowns[n])
            next_hop = neighbors[0]
            hops.append(next_hop)
            if self.knowns[next_hop] == 0:
                break
        return hops

    def crawl_from_position(self,start,backward=False):
        self.knowns = defaultdict(lambda: maxsize)
        self.knowns[start] = 0
        self.todo = PriorityQueue()
        self.todo.put((0, start))
        self.been_there = set()
        current_neighbors = self.neighbors
        if backward:
            current_neighbors = self.backward
        for todo in self.get_todo():
            if todo in self.been_there:
                continue
            self.been_there.add(todo)
            my_cost = self.knowns[todo]
            self.debug and print(f'Regarding {todo}.  Known cost is {my_cost}')
            neighbors = current_neighbors[todo]
            for neighbor, cost in neighbors.items():
                known_cost = self.knowns[neighbor]
                new_cost = cost + my_cost
                if new_cost < known_cost:
                    self.debug and print (f"IMPROVED! {neighbor} {known_cost} with {new_cost}")
                    self.knowns[neighbor] = new_cost
                    self.todo.put((new_cost, neighbor))
                else: 
                    self.debug and print (f"ALREADY GOOD! {neighbor} {known_cost} is better than {new_cost}")


In [92]:
pf = PathFinder(sample, debug=False)
pf.crawl_from_position(pf.start)

hops = pf.get_path_to_position(pf.stop)
print(len(hops))



31


In [93]:

pf = PathFinder(open('d12.input').read(), debug=False)
pf.crawl_from_position(pf.start)

hops = pf.get_path_to_position(pf.stop)
print(len(hops))


472


In [96]:
pf = PathFinder(sample, debug=False)

pf.crawl_from_position(pf.stop, backward=True)
positions = list(zip(*np.where(pf.array==97)))
positions.sort(key=lambda pos: pf.knowns[pos])
best = next(iter(positions))

pf.crawl_from_position(best)
hops = pf.get_path_to_position(pf.stop)
print(len(hops))


29


In [97]:
pf = PathFinder(open('d12.input').read(), debug=False)

pf.crawl_from_position(pf.stop, backward=True)
positions = list(zip(*np.where(pf.array==97)))
positions.sort(key=lambda pos: pf.knowns[pos])
best = next(iter(positions))

pf.crawl_from_position(best)
hops = pf.get_path_to_position(pf.stop)
print(len(hops))


465
